## ЗАДАНИЕ ДЛЯ ПРОГРАММИРУЮЩИХ

**Ссылка**, на источник текста

In [1]:
DATA_URL = "http://az.lib.ru/t/tolstoj_a_k/text_0180.shtml"

Устанавливаем библиотеки

In [2]:
import warnings
warnings.filterwarnings('ignore')
!pip -q install rnnmorph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 73.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 96.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Скачиваем текст, по которому будет дано задание, с помощью [urllib](https://docs.python.org/3/library/urllib.request.html)

In [4]:
import urllib.request

opener = urllib.request.URLopener({})

with opener.open(DATA_URL) as resource:
    raw_text = resource.read().decode(resource.headers.get_content_charset()) #Текс с html тегами

In [7]:
raw_text[:200]

'<html>\r\n<head>\r\n<title>Lib.ru/Классика: Толстой Алексей Константинович. Семья вурдалака</title>\r\n</head>\r\n\r\n<body>\r\n\r\n\r\n<center>\r\n\r\n<h2><a href=/t/tolstoj_a_k/>Толстой Алексей Константинович</a><br>\r\n'

Как видно, текст содержит html теги, от которых нужно избавиться. Выбрасываем из текста HTML-теги с помощью библиотеки [Beautiful Soap](https://beautiful-soup-4.readthedocs.io/en/latest/)

In [16]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_text, features="html.parser")

#print(soup.prettify())

In [18]:
# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
cleaned_text = soup.get_text()

In [19]:
cleaned_text[:200]

'\n\nLib.ru/Классика: Толстой Алексей Константинович. Семья вурдалака\n\n\n\nТолстой Алексей Константинович\r\nСемья вурдалака\n\n\nLib.ru/Классика:\n\r\n\n\n[Регистрация]\n \n\r\n\r\n\r\n[Найти] \r\n[Рейтинги]\r\n[Обсуждения]\r\n['

С помощью библиотеки [NLTK](https://nltk.readthedocs.io/en/stable/) разбиваем текст на предложения и токены.

In [20]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt') # download list of punctuation marks

tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(cleaned_text)]
"A total of %d 'sentences'" % len(tokenized_sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


"A total of 576 'sentences'"

Создаём объект морфологического анализатора `RNNMorph`

In [21]:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

## Задание 1
С помощью метода `str.isalpha` из стандартной библиотеки Python модифицируйте нижеследующий код так, чтобы в predictions остались только буквенные токены.

In [27]:
from tqdm import tqdm # customisable progressbar decorator for iterators

predictions = [[pred.normal_form for pred in sent if pred.normal_form.isalpha()] for sent in tqdm(predictor.predict_sentences(sentences=tokenized_sentences), "sentences") ]
predictions[-11:-10] # Избавились от токенов типа "точка", "запятая" и т.д.

9/9 [==============================] - 4s 427ms/step



sentences: 100%|██████████| 576/576 [00:00<00:00, 103772.14it/s]


[['другой',
  'ребёнок',
  'я',
  'такой',
  'же',
  'образ',
  'кинуть',
  'вслед',
  'но',
  'он',
  'упасть',
  'прямо',
  'под',
  'копыто',
  'лошадь',
  'и',
  'быть',
  'раздавленный']]

Проверьте себя. Должны получиться следующие значения:

*   Предложений: 577 (возможны расхождения в несколько предложений)
*   Токенов: примерно 8621 (возможны расхождения в некоторое количество токенов)

In [24]:
len(predictions)

576

In [25]:
non_uniq_tokens = [word for sentence in predictions for word in sentence]
len(non_uniq_tokens) 


8616

Для продолжения работы над заданием числа должны быть близки к указанным

## Задание 2

Используя `non_uniq_tokens`, стоп-слова для русского языка из библиотеки nltk (`nltk.corpus.stopwords`) и `nltk.FreqDist`, вычислите, **какую долю среди 100 самых частотных** токенов в произведении занимают токены, **не относящиеся** к стоп словам. 

**Например**, если среди 100 самых частотных слов встречается 25 слов, входящих в стоп лист, значит не входят в стоп лист 75 слов, и их доля составит 0.75. 

**Не бойтесь использовать документацию NLTK и тьюториалы.**

In [37]:
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
nltk.download("stopwords") # download list of stopwords
STOPWORDS = set(stopwords.words("russian"))
stopwords.words("russian")[:5] #Пример стоп слов

fdist = FreqDist(non_uniq_tokens)
most_common_words = [pair[0] for pair in fdist.most_common(100)]
len([word for word in most_common_words if word not in STOPWORDS]) / 100

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0.49

Проверьте себя: должно получиться 0.49 (допустимо небольшое расхождение)

## Задание 3
Вычислите, сколько токенов встречается в тексте **строго больше** 50 раз.

In [42]:
len([word for word in fdist if fdist[word] > 50])

22

Проверьте себя: должно получиться значение 22 (возможно небольшое расхождение)
